In [51]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import LabelEncoder,RobustScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,classification_report
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier


In [25]:
data=pd.read_csv("Churn.csv")
data

,Unnamed: 0,state,area.code,account.length,voice.plan,voice.messages,intl.plan,intl.mins,intl.calls,intl.charge,...,day.calls,day.charge,eve.mins,eve.calls,eve.charge,night.mins,night.calls,night.charge,customer.calls,churn
0,1,KS,area_code_415,128,yes,25,no,10.0,3,2.70,...,110,45.07,197.4,99,16.78,244.7,91,11.01,1,no
1,2,OH,area_code_415,107,yes,26,no,13.7,3,3.70,...,123,27.47,195.5,103,16.62,254.4,103,11.45,1,no
2,3,NJ,area_code_415,137,no,0,no,12.2,5,3.29,...,114,41.38,121.2,110,10.30,162.6,104,7.32,0,no
3,4,OH,area_code_408,84,no,0,yes,6.6,7,1.78,...,71,50.9,61.9,88,5.26,196.9,89,8.86,2,no
4,5,OK,area_code_415,75,no,0,yes,10.1,3,2.73,...,113,28.34,148.3,122,12.61,186.9,121,8.41,3,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,HI,area_code_408,50,yes,40,no,9.9,5,2.67,...,127,40.07,223,126,18.96,297.5,116,13.39,2,no
4996,4997,WV,area_code_415,152,no,0,no,14.7,2,3.97,...,90,31.31,256.8,73,21.83,213.6,113,9.61,3,yes
4997,4998,DC,area_code_415,61,no,0,no,13.6,4,3.67,...,89,23.9,172.8,128,14.69,212.4,97,9.56,1,no
4998,4999,DC,area_code_510,109,no,0,no,8.5,6,2.30,...,67,32.1,171.7,92,14.59,224.4,89,10.10,0,no


In [26]:
data['eve.mins']=pd.to_numeric(data['eve.mins'],errors='coerce')
data['day.charge']=pd.to_numeric(data['day.charge'],errors='coerce')

In [27]:
data.dropna(axis=0,inplace=True)
data.shape

(4969, 21)

In [28]:
data=data.drop(columns=['Unnamed: 0','state','area.code'])

In [29]:
encoder=LabelEncoder()
data['intl.plan']=encoder.fit_transform(data['intl.plan'])
data['voice.plan']=encoder.fit_transform(data['intl.plan'])
data['churn']=encoder.fit_transform(data['churn'])

In [30]:
data['Total_Charge'] =  data['day.charge'] + data['eve.charge'] + data['night.charge']
data.drop(['day.charge','eve.charge','night.charge'],axis=1,inplace=True)

In [32]:
data

,account.length,voice.plan,voice.messages,intl.plan,intl.mins,intl.calls,intl.charge,day.mins,day.calls,eve.mins,eve.calls,night.mins,night.calls,customer.calls,churn,Total_Charge
0,128,0,25,0,10.0,3,2.70,265.1,110,197.4,99,244.7,91,1,0,72.86
1,107,0,26,0,13.7,3,3.70,161.6,123,195.5,103,254.4,103,1,0,55.54
2,137,0,0,0,12.2,5,3.29,243.4,114,121.2,110,162.6,104,0,0,59.00
3,84,1,0,1,6.6,7,1.78,299.4,71,61.9,88,196.9,89,2,0,65.02
4,75,1,0,1,10.1,3,2.73,166.7,113,148.3,122,186.9,121,3,0,49.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,50,0,40,0,9.9,5,2.67,235.7,127,223.0,126,297.5,116,2,0,72.42
4996,152,0,0,0,14.7,2,3.97,184.2,90,256.8,73,213.6,113,3,1,62.75
4997,61,0,0,0,13.6,4,3.67,140.6,89,172.8,128,212.4,97,1,0,48.15
4998,109,0,0,0,8.5,6,2.30,188.8,67,171.7,92,224.4,89,0,0,56.79


In [37]:
x=data.drop(columns=['churn'])
y=data['churn']

In [42]:
scale=RobustScaler()
x=scale.fit_transform(x)
x

array([[ 0.51851852,  0.        ,  1.47058824, ..., -0.34615385,
         0.        ,  1.16008615],
       [ 0.12962963,  0.        ,  1.52941176, ...,  0.11538462,
         0.        , -0.08327351],
       [ 0.68518519,  0.        ,  0.        , ...,  0.15384615,
        -1.        ,  0.16511127],
       ...,
       [-0.72222222,  0.        ,  0.        , ..., -0.11538462,
         0.        , -0.6137832 ],
       [ 0.16666667,  0.        ,  0.        , ..., -0.42307692,
        -1.        ,  0.00646088],
       [-0.25925926,  0.        ,  2.        , ...,  0.        ,
        -1.        , -0.36109117]])

In [43]:
x,y = SMOTE().fit_resample(x,y)
y.value_counts()

0    4264
1    4264
Name: churn, dtype: int64

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=12)

In [48]:
# Define the parameters you want to tune
parameters = {
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 500, 1000, 2000]
}

# Create a XGBoost classifier
xgb_clf = xgb.XGBClassifier()

# Use GridSearchCV to find the best parameters
clf = GridSearchCV(xgb_clf, parameters, cv=5,)

# Fit the classifier to the training data
clf.fit(x_train, y_train)

# Print the best parameters
print("Best parameters found: ", clf.best_params_)

Best parameters found:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 2000}


In [50]:
xgbclf=XGBClassifier(learning_rate=0.1,max_depth=5,n_estimators=2000,random_state=42)
xgbclf.fit(x_train,y_train)
y_pred = xgbclf.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1062
           1       0.99      0.97      0.98      1070

    accuracy                           0.98      2132
   macro avg       0.98      0.98      0.98      2132
weighted avg       0.98      0.98      0.98      2132



In [57]:
parameters = {
    'criterion':['gini','entropy'],
    'max_depth': [3, 5, 7, 9],
    'n_estimators': [100, 50, 200, 500]
}

model1=RandomForestClassifier()

gscv=GridSearchCV(model1,parameters,cv=5)
gscv.fit(x_train,y_train)

#print("Best parameters found: ", gscv.best_params_)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 9],
                         'n_estimators': [100, 50, 200, 500]})

In [58]:
print("Best parameters found: ", gscv.best_params_)

Best parameters found:  {'criterion': 'entropy', 'max_depth': 9, 'n_estimators': 100}


In [52]:
model1=RandomForestClassifier(n_estimators=100,max_samples=0.8,random_state=12)
model1.fit(x_train,y_train)
y_pred=model1.predict(x_test)

print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1128
           1       0.93      0.99      0.96      1004

    accuracy                           0.96      2132
   macro avg       0.96      0.96      0.96      2132
weighted avg       0.96      0.96      0.96      2132



In [59]:
model1=RandomForestClassifier(criterion= 'entropy',n_estimators=100,max_depth=9,random_state=12)
model1.fit(x_train,y_train)
y_pred=model1.predict(x_test)

print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       1.00      0.88      0.94      1200
           1       0.87      1.00      0.93       932

    accuracy                           0.93      2132
   macro avg       0.93      0.94      0.93      2132
weighted avg       0.94      0.93      0.93      2132

